<a href="https://colab.research.google.com/github/fhfmendes04/face_recognition_4models/blob/main/MLP_Classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import cv2
from PIL import Image
from skimage import data, exposure
from skimage.feature import hog,local_binary_pattern
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

#from Models import ANN, Multinomial_LogReg

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [2]:
from sklearn.datasets import fetch_olivetti_faces
dataset = fetch_olivetti_faces(random_state=42)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [3]:
X = dataset.images
y = dataset.target

In [4]:
from collections import Counter 
d = Counter(y)
print("Quantidade de pessoas:",len(d))
print(d)

Quantidade de pessoas: 40
Counter({0: 10, 1: 10, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10, 7: 10, 8: 10, 9: 10, 10: 10, 11: 10, 12: 10, 13: 10, 14: 10, 15: 10, 16: 10, 17: 10, 18: 10, 19: 10, 20: 10, 21: 10, 22: 10, 23: 10, 24: 10, 25: 10, 26: 10, 27: 10, 28: 10, 29: 10, 30: 10, 31: 10, 32: 10, 33: 10, 34: 10, 35: 10, 36: 10, 37: 10, 38: 10, 39: 10})


# Entrada: vetor de imagem e saída(one-hot) para MLP.

In [18]:
def lbp_features(X): 
    METHOD = 'uniform'
    radius = 1
    n_points = 10
    fd_list = []
    for row in X:
        lbp_image = local_binary_pattern(row, n_points, radius, METHOD)
        fd_list.append(lbp_image.flatten())
    
    return np.array(fd_list)

In [21]:
image_lbp= lbp_features(X)
print(image_lbp.shape)

(400, 4096)


In [22]:
def hog_features(X): #====>>>>>>Usei a segunda saída da funcção que é uma imagem.
    fd_list = []
    for row in X:
        _, hog_image = hog(row, orientations=2, pixels_per_cell=(4, 4), cells_per_block=(4, 4), visualize=True)
        fd_list.append(hog_image.flatten())
    
    return np.array(fd_list)

In [23]:
image_hog = hog_features(X)
print(image_hog.shape)

(400, 4096)


In [24]:
print("HOG:",image_hog.max(), image_hog.min())
print("LBP:", image_lbp.max(), image_lbp.min())

HOG: 0.41988638043403625 0.0
LBP: 11.0 0.0


In [25]:
# transformar categorias em one-hot-encoding: Saída
saida = y.max()+1
print("Quantidade de pessoas:",saida)
target = keras.utils.to_categorical(y, saida)

Quantidade de pessoas: 40


# MLP


In [38]:
from sklearn.model_selection import KFold
from numpy.random import seed
import plotly.express as px
from sklearn.metrics import accuracy_score

In [26]:
def bissecao_rna(dEdA, dEdB, A, B, X, Yd, N):
    
    def h_linha(au, dEdA, dEdB, dv, X, Yd, N):
      An = A - au*dEdA;
      Bn = B - au*dEdB;

      dJdAn, dJdBn = calculo_gradiente(X, Yd, An, Bn, N)
      g=np.array((dJdAn.flatten(),dJdBn.flatten()))
      
      return  (np.sum(dv[0] * g[0]) + np.sum(dv[1] * g[1]))
    
    dv = np.array((-dEdA.flatten(),-dEdB.flatten())) 
    
    au = np.random.rand() # Chute inicial

    al = 0.0
    au = np.random.rand() # Chute inicial
    it = 0

    hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)

    while hl < 0: #Verifica au está acima de zero.
      al = au
      au = au * 2
      hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)
      
    a_medio = (al + au) / 2

    hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)

    itmax = np.int(np.ceil(np.log(au-al) - np.log(1e-5))/np.log(2)) #interações.

    #while (it < itmax) and ((au - al)<=1.0) and (np.abs(hl)>10.0):
    while (it < itmax):
      it += 1
      if hl > 0:
        au = a_medio
      elif hl < 0:
        al = a_medio
      elif hl == 0:
        return a_medio
      
      a_medio = (al + au) / 2
      hl = h_linha(au, dEdA, dEdB, dv, X, Yd, N)

    return a_medio

In [27]:
def calculo_saida(X, A, B, N):
  '''
    Calcula as predições para entradas usando argmax()
    X = Entrada da rede.
    A = Pesos da rede da camada do meio.
    b = Pesos da rede da camada de sáida.
    N = Número de instâncias do X.
    ________
    return: o vetor de saída com os valores das classes.
  '''
  Zin = np.dot(X, A.T)
  Z = 1/(1+np.exp(-Zin))

  Z = np.column_stack((Z,np.ones(N)))
  Yin = np.dot(Z, B.T)
  Y = 1/(1+np.exp(-Yin))

  return Y

In [28]:
def calculo_gradiente(X, Yd, A, B, N):
  Zin = np.dot(X, A.T) #(150,5)x(5xh)=(150,h)
  Z = 1/(1+np.exp(-Zin)) 

  Zb = np.column_stack((Z,np.ones(N))) #(150,h+1)
  Yin = np.dot(Zb, B.T) #(150, h+1)x(h+1,3) = (150,3) ns=3
  Y = 1/(1+np.exp(-Yin)) #(150,3)

  erro = Y -Yd 
  dg = (1-Y)*Y # (150,3) Derivada
  df = (1-Z)*Z # (150,h)
  

  dEdB = 1/N * np.dot((erro * dg).T, Zb) #(3,150)x(150,h+1)= (3,h+1)

  dEdZ = np.dot((erro* dg), B[:,:-1]) # retira o bias da miltiplicacao # (150,3)x(3,h)=(150, h)
  

  dEdA = 1/N * np.dot((dEdZ* df).T, X)  #(h,150)x(150,5)= (hx150)

  return dEdA, dEdB

In [29]:
def preditor_sigmoide(Y):
  '''
    Calcula as predições para entradas usando argmax()
    Y = relação com as saídas
    ________
    return: o vetor do resultado. 
  '''
  Y = Y.argmax(axis=1)
  return Y

In [93]:
def rna(X, Yd, h, taxa_aprendizado_fixa=None, verbose = False, epoca = 3000):
  N, ne = X.shape
  ns = Yd.shape[1]

  X = np.column_stack((X,np.ones(N)))
  seed(42)
  A = np.random.rand(h,(ne+1)) / 2  
  B = np.random.rand(ns,(h+1)) / 2  

  Y = calculo_saida(X, A, B, N)
  erro = Y - Yd


  MSE = 1/N * np.sum(erro*erro)
  
  ep = 0
  epmax = epoca
  E = 1e-2
  MSE_ant = 0
  MSE_atual = 0
  soma_MSE = 0
  
  a,b = [],[]# Utilizada no Gráfico.

  g =  np.concatenate([A.flatten(), B.flatten()])

  gradiente_norm = 1

  while (ep<epmax) and (soma_MSE < 20): #and (np.linalg.norm(g)>1e-7):
    ep += 1
    dEdA, dEdB = calculo_gradiente(X, Yd, A, B, N)

    if taxa_aprendizado_fixa is None:
      alfa = bissecao_rna(dEdA, dEdB, A, B, X, Yd, N)
    else:
      alfa = taxa_aprendizado_fixa

    A = A - alfa*dEdA
    B = B - alfa*dEdB

    #g =  np.concatenate([dEdA.flatten(), dEdB.flatten()])

    Y = calculo_saida(X, A, B, N)
    erro = Y - Yd
    
    
    MSE = 1/N * np.sum(erro*erro)

    if ((ep % 100)==0):
      print("Parâmetro :", h, "Epocas: ",ep, "MSE:", MSE, "ALFA:", alfa, 
          "Acurácia:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(Yd)),2))
    if verbose:
      a.append(ep)
      b.append(MSE)

    if MSE_ant == MSE: #Conta a quantidade de MSE iguais
      if MSE_ant == MSE_atual:
        soma_MSE +=1
      else:
        soma_MSE = 1
        MSE_atual = MSE
    else:  
      MSE_ant = MSE

  if verbose:
    fig = px.line(x= a, y=b, title='Gráfico - MLP()')
    fig.show()

  return A,B

## LBP

In [104]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(image_lbp)
x_scaler = scaler.transform(image_lbp)

In [105]:
pca = PCA(n_components=50, random_state=42)
pca.fit(x_scaler)
X_pca = pca.transform(x_scaler)

In [106]:
soma = 0
for i in range(0, len(pca.explained_variance_ratio_)):
  soma += pca.explained_variance_ratio_[i]
  print(f'PC{ i+1}: {pca.explained_variance_ratio_[i]} - {round(soma*100,2)}%')

PC1: 0.00852616691776132 - 0.85%
PC2: 0.007508200836762718 - 1.6%
PC3: 0.007210836877550287 - 2.32%
PC4: 0.006947245276997558 - 3.02%
PC5: 0.006486965282585254 - 3.67%
PC6: 0.006140078049782044 - 4.28%
PC7: 0.005945486495895884 - 4.88%
PC8: 0.005834366352587083 - 5.46%
PC9: 0.0056727536328024745 - 6.03%
PC10: 0.005612148204464098 - 6.59%
PC11: 0.005393190610877594 - 7.13%
PC12: 0.005360359898486351 - 7.66%
PC13: 0.005236744640906278 - 8.19%
PC14: 0.005118741933208122 - 8.7%
PC15: 0.00510838364907438 - 9.21%
PC16: 0.004917697550830504 - 9.7%
PC17: 0.004901386614667453 - 10.19%
PC18: 0.0048510048000801505 - 10.68%
PC19: 0.004798971686521533 - 11.16%
PC20: 0.004684729044474385 - 11.63%
PC21: 0.004642511675488156 - 12.09%
PC22: 0.004624989078216922 - 12.55%
PC23: 0.0045980029531676745 - 13.01%
PC24: 0.004513707436238474 - 13.46%
PC25: 0.004451966184854102 - 13.91%
PC26: 0.00443816812234517 - 14.35%
PC27: 0.004391811193293804 - 14.79%
PC28: 0.004322152335132418 - 15.22%
PC29: 0.004304684580

In [107]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(X_pca)
X_pca = scaler.transform(X_pca)

In [108]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.30, random_state=42,stratify=target)

In [109]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((280, 50), (280, 40), (120, 50), (120, 40))

In [110]:
# Treino com melhor Parâmetro 
A, B = rna(x_train, y_train, 150, epoca=5000)

#Teste 
x_test_one = np.column_stack((x_test,np.ones(x_test.shape[0])))
Y = calculo_saida(x_test_one, A, B, x_test.shape[0])
print("Acurácia do Teste LBP:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_test)),2))

Parâmetro : 150 Epocas:  100 MSE: 0.7044532647399964 ALFA: 32.776183559752255 Acurácia: 0.32
Parâmetro : 150 Epocas:  200 MSE: 0.5764895117818409 ALFA: 19.483938479998486 Acurácia: 0.43
Parâmetro : 150 Epocas:  300 MSE: 0.49276110062718875 ALFA: 94.16830021905247 Acurácia: 0.51
Parâmetro : 150 Epocas:  400 MSE: 0.449220221204481 ALFA: 210.1175696194703 Acurácia: 0.55
Parâmetro : 150 Epocas:  500 MSE: 0.39663501532514334 ALFA: 1141.0450935529457 Acurácia: 0.6
Parâmetro : 150 Epocas:  600 MSE: 0.379365245268344 ALFA: 132.55706978694315 Acurácia: 0.62
Parâmetro : 150 Epocas:  700 MSE: 0.3288598676953171 ALFA: 93.2253872663624 Acurácia: 0.68
Parâmetro : 150 Epocas:  800 MSE: 0.29643200026721994 ALFA: 20436.678820712244 Acurácia: 0.7
Parâmetro : 150 Epocas:  900 MSE: 0.2607147665236186 ALFA: 2161500.5517211687 Acurácia: 0.74
Parâmetro : 150 Epocas:  1000 MSE: 0.24285776858420094 ALFA: 33314.915457765164 Acurácia: 0.76
Parâmetro : 150 Epocas:  1100 MSE: 0.2121632030147829 ALFA: 63.5029551474

## HOG

In [96]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(image_hog)
x_scaler = scaler.transform(image_hog)

In [97]:
pca = PCA(n_components=50, random_state=42)
pca.fit(x_scaler)
X_pca = pca.transform(x_scaler)

In [98]:
#Normalização Z-score
scaler = StandardScaler()
scaler.fit(X_pca)
X_pca = scaler.transform(X_pca)

In [99]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, target, test_size=0.30, random_state=42,stratify=target)

In [100]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((280, 50), (280, 40), (120, 50), (120, 40))

In [102]:
# Treino com melhor Parâmetro 
A, B = rna(x_train, y_train, 100, epoca=3000)

#Teste 
x_test_one = np.column_stack((x_test,np.ones(x_test.shape[0])))
Y = calculo_saida(x_test_one, A, B, x_test.shape[0])
print("Acurácia do Teste HOG:",round(accuracy_score(preditor_sigmoide(Y), preditor_sigmoide(y_test)),2))

Parâmetro : 100 Epocas:  100 MSE: 0.4179872295271163 ALFA: 125.3837255648919 Acurácia: 0.61
Parâmetro : 100 Epocas:  200 MSE: 0.3630697155770572 ALFA: 455.543800039418 Acurácia: 0.65
Parâmetro : 100 Epocas:  300 MSE: 0.33109026177060386 ALFA: 22206.41698822943 Acurácia: 0.67
Parâmetro : 100 Epocas:  400 MSE: 0.3036533289444122 ALFA: 7671.983767672762 Acurácia: 0.71
Parâmetro : 100 Epocas:  500 MSE: 0.27149957259290525 ALFA: 2833.559348935952 Acurácia: 0.73
Parâmetro : 100 Epocas:  600 MSE: 0.23285744786345225 ALFA: 20.611530676081472 Acurácia: 0.77
Parâmetro : 100 Epocas:  700 MSE: 0.19440023329235875 ALFA: 183.95163500426094 Acurácia: 0.81
Parâmetro : 100 Epocas:  800 MSE: 0.18571581933711553 ALFA: 255819.27141928067 Acurácia: 0.81
Parâmetro : 100 Epocas:  900 MSE: 0.1608608900048361 ALFA: 1661.3354460758783 Acurácia: 0.84
Parâmetro : 100 Epocas:  1000 MSE: 0.16071433418845144 ALFA: 24812856.147390112 Acurácia: 0.84
Parâmetro : 100 Epocas:  1100 MSE: 0.1464285908674316 ALFA: 40619931.